In [12]:
!pip install weave

In [13]:
import json
import openai
import weave

class ExtractFruitsModel(weave.Model):
    model_name: str
    prompt_template: str

    @weave.op()
    async def predict(self, sentence: str) -> dict:
        client = openai.AsyncClient()

        response = await client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "user", "content": self.prompt_template.format(sentence=sentence)}
            ],
        )
        result = response.choices[0].message.content
        if result is None:
            raise ValueError("No response from model")
        parsed = json.loads(result)
        return parsed

In [18]:
import asyncio
import weave

# Import Colab Secrets userdata module
from google.colab import userdata

# Set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

weave.init('intro-example')

model = ExtractFruitsModel(model_name='gpt-3.5-turbo-1106',
                        prompt_template='Extract fields ("fruit": <str>, "color": <str>, "flavor": <str>) from the following text, as json: {sentence}')
sentence = "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy."
# print(asyncio.run(model.predict(sentence)))
# if you're in a Jupyter Notebook, run:
await model.predict(sentence)

/usr/local/lib/python3.10/dist-packages/openai/_response.py:684: RuntimeWarning: coroutine 'Evaluation.evaluate' was never awaited
  def wrapped(*args: P.args, **kwargs: P.kwargs) -> AsyncResponseContextManager[AsyncAPIResponse[R]]:


🍩 https://wandb.ai/simran_anand/intro-example/r/call/01944f89-d326-78b2-b35c-253d9c3e4055


{'fruit': 'neoskizzles', 'color': 'purple', 'flavor': 'taste like candy'}

In [19]:
sentences = [
    "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy.",
    "Pounits are a bright green color and are more savory than sweet.",
    "Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."
]
labels = [
    {'fruit': 'neoskizzles', 'color': 'purple', 'flavor': 'candy'},
    {'fruit': 'pounits', 'color': 'bright green', 'flavor': 'savory'},
    {'fruit': 'glowls', 'color': 'pale orange', 'flavor': 'sour and bitter'}
]
examples = [
    {'id': '0', 'sentence': sentences[0], 'target': labels[0]},
    {'id': '1', 'sentence': sentences[1], 'target': labels[1]},
    {'id': '2', 'sentence': sentences[2], 'target': labels[2]}
]

In [21]:
import weave
from weave.scorers import MultiTaskBinaryClassificationF1

weave.init('intro-example')

@weave.op()
def fruit_name_score(target: dict, output: dict) -> dict:
    return {'correct': target['fruit'] == output['fruit']}

evaluation = weave.Evaluation(
    dataset=examples,
    scorers=[
        MultiTaskBinaryClassificationF1(class_names=["fruit", "color", "flavor"]),
        fruit_name_score
    ],
)
# print(asyncio.run(evaluation.evaluate(model)))
# if you're in a Jupyter Notebook, run:
await evaluation.evaluate(model)

Evaluated 1 of 3 examples

Evaluated 2 of 3 examples

Evaluated 3 of 3 examples

Evaluation summary
{
    'MultiTaskBinaryClassificationF1': {
        'fruit': {'f1': 0.8, 'precision': 0.6666666666666666, 'recall': 1.0},
        'color': {'f1': 1.0, 'precision': 1.0, 'recall': 1.0},
        'flavor': {'f1': 0.5, 'precision': 0.3333333333333333, 'recall': 1.0}
    },
    'fruit_name_score': {'correct': {'true_count': 2, 'true_fraction': 0.6666666666666666}},
    'model_latency': {'mean': 1.017319122950236}
}

🍩 https://wandb.ai/simran_anand/intro-example/r/call/01944f8a-37bd-7972-a19d-5228d840dbd7


{'MultiTaskBinaryClassificationF1': {'fruit': {'f1': 0.8,
   'precision': 0.6666666666666666,
   'recall': 1.0},
  'color': {'f1': 1.0, 'precision': 1.0, 'recall': 1.0},
  'flavor': {'f1': 0.5, 'precision': 0.3333333333333333, 'recall': 1.0}},
 'fruit_name_score': {'correct': {'true_count': 2,
   'true_fraction': 0.6666666666666666}},
 'model_latency': {'mean': 1.017319122950236}}

In [28]:
import wandb
import weave

# Initialize the W&B run and log outputs (as before)
wandb.init(project='llm-output-comparison')

# Simulate some outputs from two LLMs
llm_1_outputs = [
    "The quick brown fox jumps over the lazy dog.",
    "A journey of a thousand miles begins with a single step.",
    "To be or not to be, that is the question."
]

llm_2_outputs = [
    "In the middle of difficulty lies opportunity.",
    "Life is 10% what happens to us and 90% how we react to it.",
    "The only impossible journey is the one you never begin."
]

# Log these outputs to W&B
wandb.log({
    "llm_1_outputs": llm_1_outputs,
    "llm_2_outputs": llm_2_outputs
})

wandb.finish()

# Fetch the logged data using wandb.Api()
api = wandb.Api()

# Fetch the specific run by its ID (replace this with your actual run ID)
run = api.run("simran_anand/llm-output-comparison/7245ncd2")  # replace 'run-id' with the actual run ID

# Access the logged data from the run
llm_1_outputs = run.summary.get("llm_1_outputs", [])
llm_2_outputs = run.summary.get("llm_2_outputs", [])

# Use Weave to visualize the outputs
print(llm_1_outputs)

print(llm_2_outputs)


True
['The quick brown fox jumps over the lazy dog.', 'A journey of a thousand miles begins with a single step.', 'To be or not to be, that is the question.']
['In the middle of difficulty lies opportunity.', 'Life is 10% what happens to us and 90% how we react to it.', 'The only impossible journey is the one you never begin.']


In [27]:
import weave
from openai import OpenAI

client = OpenAI()

# Weave will track the inputs, outputs and code of this function
@weave.op()
def extract_dinos(sentence: str) -> dict:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """In JSON format extract a list of `dinosaurs`, with their `name`,
their `common_name`, and whether its `diet` is a herbivore or carnivore"""
            },
            {
                "role": "user",
                "content": sentence
            }
            ],
            response_format={ "type": "json_object" }
        )
    return response.choices[0].message.content


# Initialise the weave project
weave.init('jurassic-park')

sentence = """I watched as a Tyrannosaurus rex (T. rex) chased after a Triceratops (Trike), \
both carnivore and herbivore locked in an ancient dance. Meanwhile, a gentle giant \
Brachiosaurus (Brachi) calmly munched on treetops, blissfully unaware of the chaos below."""

result = extract_dinos(sentence)
print(result)

Logged in as Weights & Biases user: simran_anand.
View Weave data at https://wandb.ai/simran_anand/jurassic-park/weave
🍩 https://wandb.ai/simran_anand/jurassic-park/r/call/01944f98-89f6-7402-ab5a-e079c6f81a79
{
  "dinosaurs": [
    {
      "name": "Tyrannosaurus rex",
      "common_name": "T. rex",
      "diet": "carnivore"
    },
    {
      "name": "Triceratops",
      "common_name": "Trike",
      "diet": "herbivore"
    },
    {
      "name": "Brachiosaurus",
      "common_name": "Brachi",
      "diet": "herbivore"
    }
  ]
}


In [33]:
from openai import OpenAI
import weave
from weave import Model
import numpy as np
import json
import asyncio

# Examples we've gathered that we want to use for evaluations
articles = [
    "Novo Nordisk and Eli Lilly rival soars 32 percent after promising weight loss drug results Shares of Denmarks Zealand Pharma shot 32 percent higher in morning trade, after results showed success in its liver disease treatment survodutide, which is also on trial as a drug to treat obesity. The trial “tells us that the 6mg dose is safe, which is the top dose used in the ongoing [Phase 3] obesity trial too,” one analyst said in a note. The results come amid feverish investor interest in drugs that can be used for weight loss.",
    "Berkshire shares jump after big profit gain as Buffetts conglomerate nears $1 trillion valuation Berkshire Hathaway shares rose on Monday after Warren Buffetts conglomerate posted strong earnings for the fourth quarter over the weekend. Berkshires Class A and B shares jumped more than 1.5%, each. Class A shares are higher by more than 17% this year, while Class B has gained more than 18%. Berkshire was last valued at $930.1 billion, up from $905.5 billion where it closed on Friday, according to FactSet. Berkshire on Saturday posted fourth-quarter operating earnings of $8.481 billion, about 28 percent higher than the $6.625 billion from the year-ago period, driven by big gains in its insurance business. Operating earnings refers to profits from businesses across insurance, railroads and utilities. Meanwhile, Berkshires cash levels also swelled to record levels. The conglomerate held $167.6 billion in cash in the fourth quarter, surpassing the $157.2 billion record the conglomerate held in the prior quarter.",
    "Highmark Health says its combining tech from Google and Epic to give doctors easier access to information Highmark Health announced it is integrating technology from Google Cloud and the health-care software company Epic Systems. The integration aims to make it easier for both payers and providers to access key information they need, even if it's stored across multiple points and formats, the company said. Highmark is the parent company of a health plan with 7 million members, a provider network of 14 hospitals and other entities",
    "Rivian and Lucid shares plunge after weak EV earnings reports Shares of electric vehicle makers Rivian and Lucid fell Thursday after the companies reported stagnant production in their fourth-quarter earnings after the bell Wednesday. Rivian shares sank about 25 percent, and Lucids stock dropped around 17 percent. Rivian forecast it will make 57,000 vehicles in 2024, slightly less than the 57,232 vehicles it produced in 2023. Lucid said it expects to make 9,000 vehicles in 2024, more than the 8,428 vehicles it made in 2023.",
    "Mauritius blocks Norwegian cruise ship over fears of a potential cholera outbreak Local authorities on Sunday denied permission for the Norwegian Dawn ship, which has 2,184 passengers and 1,026 crew on board, to access the Mauritius capital of Port Louis, citing “potential health risks.” The Mauritius Ports Authority said Sunday that samples were taken from at least 15 passengers on board the cruise ship. A spokesperson for the U.S.-headquartered Norwegian Cruise Line Holdings said Sunday that 'a small number of guests experienced mild symptoms of a stomach-related illness' during Norwegian Dawns South Africa voyage.",
    "Intuitive Machines lands on the moon in historic first for a U.S. company Intuitive Machines Nova-C cargo lander, named Odysseus after the mythological Greek hero, is the first U.S. spacecraft to soft land on the lunar surface since 1972. Intuitive Machines is the first company to pull off a moon landing — government agencies have carried out all previously successful missions. The company's stock surged in extended trading Thursday, after falling 11 percent in regular trading.",
    "Lunar landing photos: Intuitive Machines Odysseus sends back first images from the moon Intuitive Machines cargo moon lander Odysseus returned its first images from the surface. Company executives believe the lander caught its landing gear sideways on the surface of the moon while touching down and tipped over. Despite resting on its side, the company's historic IM-1 mission is still operating on the moon.",
]

def docs_to_embeddings(docs: list) -> list:
    openai = OpenAI()
    document_embeddings = []
    for doc in docs:
        response = (
            openai.embeddings.create(input=doc, model="text-embedding-3-small")
            .data[0]
            .embedding
        )
        document_embeddings.append(response)
    return document_embeddings

article_embeddings = docs_to_embeddings(articles) # Note: you would typically do this once with your articles and put the embeddings & metadata in a database

# We've added a decorator to our retrieval step
@weave.op()
def get_most_relevant_document(query):
    openai = OpenAI()
    query_embedding = (
        openai.embeddings.create(input=query, model="text-embedding-3-small")
        .data[0]
        .embedding
    )
    similarities = [
        np.dot(query_embedding, doc_emb)
        / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in article_embeddings
    ]
    # Get the index of the most similar document
    most_relevant_doc_index = np.argmax(similarities)
    return articles[most_relevant_doc_index]

# We create a Model subclass with some details about our app, along with a predict function that produces a response
class RAGModel(Model):
    system_message: str
    model_name: str = "gpt-3.5-turbo-1106"

    @weave.op()
    def predict(self, question: str) -> dict: # note: `question` will be used later to select data from our evaluation rows
        from openai import OpenAI
        context = get_most_relevant_document(question)
        client = OpenAI()
        query = f"""Use the following information to answer the subsequent question. If the answer cannot be found, write "I don't know."
        Context:
        \"\"\"
        {context}
        \"\"\"
        Question: {question}"""
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": query},
            ],
            temperature=0.0,
            response_format={"type": "text"},
        )
        answer = response.choices[0].message.content
        return {'answer': answer, 'context': context}

weave.init('rag-qa')
model = RAGModel(
    system_message="You are an expert in finance and answer questions related to finance, financial services, and financial markets. When responding based on provided information, be sure to cite the source."
)

# Here is our scoring function uses our question and output to product a score
@weave.op()
async def context_precision_score(question, output):
    context_precision_prompt = """Given question, answer and context verify if the context was useful in arriving at the given answer. Give verdict as "1" if useful and "0" if not with json output.
    Output in only valid JSON format.

    question: {question}
    context: {context}
    answer: {answer}
    verdict: """
    client = OpenAI()

    prompt = context_precision_prompt.format(
        question=question,
        context=output['context'],
        answer=output['answer'],
    )

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" }
    )
    response_message = response.choices[0].message
    response = json.loads(response_message.content)
    return {
        "verdict": int(response["verdict"]) == 1,
    }

questions = [
    {"question": "What significant result was reported about Zealand Pharma's obesity trial?"},
    {"question": "How much did Berkshire Hathaway's cash levels increase in the fourth quarter?"},
    {"question": "What is the goal of Highmark Health's integration of Google Cloud and Epic Systems technology?"},
    {"question": "What were Rivian and Lucid's vehicle production forecasts for 2024?"},
    {"question": "Why was the Norwegian Dawn cruise ship denied access to Mauritius?"},
    {"question": "Which company achieved the first U.S. moon landing since 1972?"},
    {"question": "What issue did Intuitive Machines' lunar lander encounter upon landing on the moon?"}
]

# We define an Evaluation object and pass our example questions along with scoring functions
evaluation = weave.Evaluation(dataset=questions, scorers=[context_precision_score])
await evaluation.evaluate(model)

/usr/lib/python3.10/tokenize.py:527: RuntimeWarning: coroutine 'ExtractFruitsModel.predict' was never awaited
  pseudomatch = _compile(PseudoToken).match(line, pos)
/usr/lib/python3.10/tokenize.py:527: RuntimeWarning: coroutine 'Evaluation.evaluate' was never awaited
  pseudomatch = _compile(PseudoToken).match(line, pos)


🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-b661-7913-952d-cbdd739e62df
🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-b874-76b0-b43d-15daea698bdb
🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-ba1e-7cb2-b16f-58b5a2582160
🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-bb35-7e52-9c44-caec14b2b877
🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-bc38-72d3-81b0-c65087e89b65
🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-bd8b-7560-9bbd-d3a3c23c67d2
🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-be97-7051-94b0-d73a1f08c91f


Evaluated 1 of 7 examples

Evaluated 2 of 7 examples

Evaluated 3 of 7 examples

...

Evaluated 6 of 7 examples

Evaluated 7 of 7 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 7, 'true_fraction': 1.0}},
    'model_latency': {'mean': 26.492431231907435}
}

🍩 https://wandb.ai/simran_anand/rag-qa/r/call/01944fa1-c00e-73f0-aa79-696c8de443c9


{'context_precision_score': {'verdict': {'true_count': 7,
   'true_fraction': 1.0}},
 'model_latency': {'mean': 26.492431231907435}}